<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/DyFor%20GP%20Heuristics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import math
import plotly.graph_objects as go
import keras
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed, Input
from keras.models import Model
from keras import saving

import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
import plotly
from keras import losses
import plotly.express as px # for data visualization
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
import numpy as np
import scipy.stats as stats
import pylab as pl



##### Take one dataset at a time
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
#train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/multivariate_long_sequences-TRAIN-Daily-DIRECT-VAR.npy')
train_data = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/multivariate_long_sequences-TRAIN-DATA-FullSearch-VAR.npy')


n_seq = train_data.shape[0]
window_size = train_data.shape[1]
n_features = train_data.shape[2]
TestSample = 1


##Get large set of subsequence from previous steps (VAE)... this is not labelled yet.
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated_large_subsquence2_data_V2.npy')
#results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated_large_subsquence2_data.npy')
results1=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated_large_subsquence2_data.npy')

x=results1

import random
rd = list()
rd = random.sample(range(1, results1.shape[0]), TestSample)


############### VAR PREDICTION TIMING ####################################################################################################################

!pip install ipython-autotime
##Start measuring time-----THIS ONE IS FOR VAR --------------------------------
%load_ext autotime

rmse_list = list()
min_window_list = list()
best_window_for_long_seq = list()
best_window_for_long_seq.clear()
#for i in range(2) :
x_3d = x.reshape((x.shape[0],window_size,n_features))
n_future = 1
K= window_size



best_window_for_long_seq.clear()

from statsmodels.tsa.api import VAR

## Note for some reason first run takes longer. So run it twice to get a better time
for i in rd :

  rmse_list.clear()
  for k in range(2,(round(K))):
    cur_seq = x_3d[i,:,:]
    #df = pd.DataFrame(cur_seq, columns=['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12'])
    df = pd.DataFrame(cur_seq, columns=['V1','V2','V3','V4','V5','V6'])
    df_train, df_test = df[0:-n_future], df[-n_future:]
    model= VAR(df_train)
    try:
      model_fitted1 = model.fit(k)
      forecast_input1 = df_train.values[-k:]
      fc1 = model_fitted1.forecast(y=forecast_input1, steps=n_future)
      df_forecast1 = pd.DataFrame(fc1, index=df.index[-n_future:], columns=df.columns)
      mse =  mean_squared_error(df_test['V1'], df_forecast1['V1'].values)
      rmse_list.append(mse)
    except:
      rmse_list.append(99999)
      print('VAR could not solve row number')
      print(i, k)

  #For this i, find minimum rmse for all sliding window, and corresponding sw size
  min_index = rmse_list.index(min(rmse_list))
  min_sw = min_index + 2
  print('i=', i,'SW =', min_sw, rmse_list)
  best_window_for_long_seq.append(min_sw)


#OUT OF ALL LOOPS NOW
#best_window_for_long_seq now contains the best multivariate window size for each of the long sequence i
print(best_window_for_long_seq)
Window = np.array(best_window_for_long_seq)
y= Window

##Commenting this for now but they may be needed
#np.save(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data-true-window2-COHORT1.npy',y)
#np.save(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data2-COHORT1.npy',x)


########################## MLP PREDICTION TIMING ###################################################################################
#---------------------------------------------------------Get parts of the results and combine them --------------------------------


#y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data-true-window2.npy')
#x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_METROPM/generated-data2.npy')

#y = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data-true-window2.npy')
#x = np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_AIRQUALITY/generated-data2.npy')

x=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data2.npy')
y=np.load(r'/content/drive/MyDrive/PHD/2024/TEMP_OUTPUT_ELECTRICITY/generated-data-true-window2.npy')

from sklearn.preprocessing import PowerTransformer
transformer = StandardScaler()
y_transformed = transformer.fit_transform(y.reshape(-1,1)).flatten()

##Simply take first 100 records - no need to sample... we just need to check prediction time

x_test = x[TestSample:TestSample+1,:]
y_test = y_transformed[TestSample:TestSample + 1]
#------------------------------------------------------Retrieve Trained MLP Model----------

#base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/DGRNet-MLP-Versions/METROPM_MLP_model_Daily.keras')
#base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/DGRNet-MLP-Versions/AQ_MLP_model_Daily.keras')
base_model = keras.models.load_model(r'/content/drive/MyDrive/PHD/2024/DGRNet-MLP-Versions/ELEC_MLP_model_Daily.keras')

#-----------------------------------------------------Run Prediction on MLP------------------------
## Note for some reason first run takes longer. So run it twice to get a better time

y_pred_raw = base_model.predict(x_test)
y_test_pred = transformer.inverse_transform(y_pred_raw)
print(y_test_pred)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
time: 129 ms (started: 2025-03-18 16:01:00 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')